<a href="https://colab.research.google.com/github/NathWolf/NATCOR_heuropt/blob/main/NATCOR_Challenge_1_MILP_and_complexity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to MILP and Gurobi

MILP is a mathematical language for modelling optimisation problems.
Each 'program' (or formulation) consists of a set of decision variables, constraints (equalities or inequalities) and an objective function (a formula).
Decision variables can be continuous or integer (hence the name of the language).
Most commonly, we use binary variables, i.e. integer variables with only two possible values: 0 and 1.
The constraints and the objective function use decision variables in arbitrary formulas.
However, for a program to be linear, we need to ensure that the constraints and the objective function have to be linear.
Solvers such as CPLEX and Gurobi are primarily designed for linear programs.

To develop an MILP model, you need to follow these steps:
1. Decide on the decision variables.
 Decision variables should encode a solution to your problem.
 For the shortest path problem, the solution is a path, hence they should encode a path within the given graph.
 Once you know the values of all the decision variables, you should be able to restore the solution (the path in our example).
 Also, you should be able to encode any feasible solution to your problem with those decision variables.

 This is arguably the hardest part of the task.

1. Define the constraints via the decision variable.
 Assuming that the decision variables that you defined in the first step are $x_1$, $x_2$, ..., $x_{100}$, the following is an example of a constraint:
 $$
 x_5 - 3 x_{12} \le 40.
 $$
 By defining such a constraint, you request that $x_5 - 3 x_{12}$ should never exceed 40.
 Here is another example, more relevant to the shortest path problem:
 $$
 x_1 + x_2 + x_3 + x_4 + x_5 = 1.
 $$
 If the $x$-variables are binary then the above constraint requests that exactly one of the five variables has value 1 and the rest of them should have value 0.

1. Define the objective.
 The objective can be to minimise something or to maximise something.
 As the shortest path problem is a minimisation problem, you will need to minimise the objective function.
 An example of an objective:
 $$
 \text{minimise } 4 x_3 - x_{15}.
 $$

Note that MILP is a declarative language; you describe the problem – not the algorithm.
Specifically, using a mathematical language, you describe a solution (what decision variables encode it), the constraints (what makes a solution feasible) and the objective function (what needs to be minimised or maximised), and then the solver automatically does the job for you.
This is a very powerful approach to solving optimisation problems, and this is how most of the optimisation problems are solved outside academia.

We will use Gurobi – the state-of-the-art mixed integer programming solver.  It is a commercial software system but it is free for a small models or academic use.  

To install the Gurobi package, run the below cell.

In [ ]:
!pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 42.1 MB/s eta 0:00:00


Below is an example of a script that creates a simple MILP model and solves it with Gurobi.  It solves the following problem: find the largest five-digit binary number with exactly two non-zero digits.  For example, $01001_2 = 9_{10}$ is a feasible solution to this problem but it is not the optimal solution.  Also, $11111_2 = 31_{10}$ is not a feasible solution as the number of non-zero digits is greater than 2.

To formalise this problem, we will introduce five binary decision variables: $x_0$, $x_1$, $x_2$, $x_3$, $x_4$.  Each of them correspond to one digit in our number.  Knowing the values of these variables, we can calculate the number (let us call it $v$):
$$
v = 1 \cdot x_0 + 2 \cdot x_1 + 4 \cdot x_2 + 8 \cdot x_3 + 16 \cdot x_4.
$$

We also have a constraint: the number of non-zero digits in $v$ should be exactly 2:
$$
x_0 + x_1 + x_2 + x_3 + x_4 = 2.
$$

Thus, our formulation of the problem is as follows:
$$
\begin{align}
& \text{Maximise } \sum_{i = 0}^4 2^i \cdot x_i, \\
& \text{subject to } \\
& \qquad \sum_{i=0}^4 x_i = 2, \\
& \qquad x_i \in \{ 0, 1 \} \qquad \forall i \in \{0, 1, 2, 3, 4\}.
\end{align}
$$

We can now convert this formulation into a Python script.

In [ ]:
from gurobipy import *

# Create an empty model:
model = Model()

# x is an array of five binary decision variables; during the solution process,
# Gurobi will decide what value to assign to each of them: 0 or 1
x = model.addVars(5, vtype=GRB.BINARY)

# This constraint requests that exactly two of them should be assigned value 1:
model.addConstr(x.sum('*') == 2)

# Setting the objective:
model.setObjective(quicksum(2**i * x[i] for i in range(5)), GRB.MAXIMIZE)

# An equivalent code without a loop:
# model.setObjective(x[0] + 2*x[1] + 4*x[2] + 8*x[3] + 16*x[4], GRB.MAXIMIZE)

# Run the solver:
model.optimize()

if model.Status == GRB.OPTIMAL:  # Checking if the solver found a solution
    # If it found a solution, print it:
    v = 0
    for i in range(4, -1, -1):
        print(f'The value of x[{i}] is {x[i].X}')
        v += 2**i * x[i].X  # Computing v

    print(f'Decimal representation: {v}')
else:
    print('No feasible solution found')

# Task 1.1 (see the PDF for details)

Please write your proof below.

Your proof...

# Task 1.2 (see the PDF for details)

Please write your MILP formulation of the problem below.  Note that you can use LaTeX commands in Google Colab to typeset formulas.

Your formulation...

# Task 1.3 (see the PDF for details)

You are provided with implementations of the `Instance` and `Solution` classes.  An `Instance` object stores all the instance data: the values $n$ and $k$ and the edge weights.  We assume that the graph is complete (every node is connected to every other node); if some edge is not available, one can set its weight to a very large number effectively eliminating it from solutions.

The source node is always node 0; the destination node is always node $n - 1$.

The weights are stored in a three-dimensional matrix `weights`.  The first index is the 'from node', the second index is the 'to node', and the third index is the scenario index.

In [ ]:
import numpy as np

class Instance:
    def __init__(self, n, k, seed):
        self.n = n
        self.k = k
        np.random.seed = seed
        self.weights = np.random.randint(0, 100, size=(n, n, k))

        # 'Remove' loops:
        for i in range(n):
            self.weights[i, i, :] = 100000


class Solution:
    # Creates a new solution object.  'path' is a list of integers, giving
    # the sequence of nodes in the path.
    def __init__(self, instance, path):
        self.instance = instance
        self.path = path
        assert self.is_feasible()

    def is_feasible(self):
        # A path cannot be empty:
        if len(self.path) == 0:
            return False

        # Check the first and the last nodes:
        if self.path[0] != 0 or self.path[-1] != self.instance.n - 1:
            return False

        # No repeatitions allowed:
        if len(self.path) != len(set(range(len(self.path)))):
            return False

        return True

    # Calculates and returns the objective value:
    def get_objective(self):
        return max(sum(self.instance.weights[self.path[i], self.path[i+1], l]
                   for i in range(len(self.path) - 1))
                for l in range(self.instance.k))

    def __str__(self):
        return ' -> '.join(str(node) for node in self.path) \
            + f'; objective: {self.get_objective()}'

Your main task is to implement a Gurobi-based solver for the Absolute Robust Shortest Path problem.

1. Start by implementing the solver for the case of one scenario.  This is the standard shortest path problem.  _Hint:_ start by producing a formulation of the problem, similar to the formulas in the example above.  Use pen and paper.  You will need to decide how to represent a solution to the problem using decision variables.  You might be tempted to use integer variables that can can take many values but usually it is easier to use binary variables to represent solutions.  Once you know your decision variables, formulate the constraints to forbid infeasible solutions.  Then define the objective.

2. Extend your implementation to the cases with multiple scenarios ($k > 1$).  Test it.

3. Optional.  Can you produce 'hard instances', i.e. instances that make the solver slow?  Note that you cannot significantly increase the size of the instances because the free version of Gurobi limits the maximum size of the model (to solve large models, you would need to apply for a free academic licence).

In [ ]:
from gurobipy import *

def solve(instance):
    model = Model()

    # Create decision variables

    # Define the constraints

    # Set the objective

    if model.Status == GRB.OPTIMAL:
        print('Solution found')

        # Extract the path from Gurobi
        path = ...

        solution = Solution(instance, path)
        print(solution)
    else:
        print('No solution')